In [ ]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File ‘edgar_allan_poe.txt’ already there; not retrieving.

File ‘robert_frost.txt’ already there; not retrieving.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [ ]:
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

In [ ]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [ ]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [ ]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [ ]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [ ]:
len(Ytrain), len(Ytest)

(1615, 539)

In [ ]:
train_text[:5]

['rather than tip a table for you let me',
 'mother yes we could too son tell the truth for once',
 'you must be learned thats what you see in it',
 'who alterest all things with thy peering eyes',
 'they found hed gnawed the four posts of his bed']

In [ ]:
Ytrain[:5]

[1, 1, 1, 0, 1]

In [ ]:
idx = 1
word2idx = {'<unk>': 0}

In [ ]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [ ]:
word2idx

{'<unk>': 0,
 'rather': 1,
 'than': 2,
 'tip': 3,
 'a': 4,
 'table': 5,
 'for': 6,
 'you': 7,
 'let': 8,
 'me': 9,
 'mother': 10,
 'yes': 11,
 'we': 12,
 'could': 13,
 'too': 14,
 'son': 15,
 'tell': 16,
 'the': 17,
 'truth': 18,
 'once': 19,
 'must': 20,
 'be': 21,
 'learned': 22,
 'thats': 23,
 'what': 24,
 'see': 25,
 'in': 26,
 'it': 27,
 'who': 28,
 'alterest': 29,
 'all': 30,
 'things': 31,
 'with': 32,
 'thy': 33,
 'peering': 34,
 'eyes': 35,
 'they': 36,
 'found': 37,
 'hed': 38,
 'gnawed': 39,
 'four': 40,
 'posts': 41,
 'of': 42,
 'his': 43,
 'bed': 44,
 'greenest': 45,
 'our': 46,
 'valleys': 47,
 'tis': 48,
 'not': 49,
 'that': 50,
 'flowers': 51,
 'twenty': 52,
 'springs': 53,
 'and': 54,
 'since': 55,
 'there': 56,
 'was': 57,
 'no': 58,
 'other': 59,
 'way': 60,
 'to': 61,
 'look': 62,
 'several': 63,
 'times': 64,
 'over': 65,
 'by': 66,
 'marrying': 67,
 'father': 68,
 'but': 69,
 'though': 70,
 'runs': 71,
 'unbridled': 72,
 'off': 73,
 'its': 74,
 'course': 75,
 'gro

In [ ]:
len(word2idx)

2510

In [ ]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [ ]:
train_text_int[100:105]

[[42, 19, 386],
 [27, 57, 79, 66, 17, 300, 301, 42, 188],
 [123, 302, 303, 191, 304, 69, 305],
 [74, 32, 197, 26, 17, 387, 70, 74, 17, 105],
 [388, 389, 17, 390, 123, 276, 195, 391, 257]]

In [ ]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [ ]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [ ]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [ ]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [ ]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.3269349845201238, 0.6730650154798762)

In [ ]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]

      # update last_idx
      last_idx = idx

    return logprob

  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [ ]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [ ]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9944272445820433


In [ ]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8237476808905381


In [ ]:
from sklearn.metrics import confusion_matrix, f1_score

# read about F-score: https://en.wikipedia.org/wiki/F-score

In [ ]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 519,    9],
       [   0, 1087]])

In [ ]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[104,  86],
       [  9, 340]])

In [ ]:
f1_score(Ytrain, Ptrain)

0.9958772331653688

In [ ]:
f1_score(Ytest, Ptest)

0.8774193548387097